In [ ]:
import datasets
print(datasets.__version__)


4.0.0


In [ ]:
!pip uninstall -y datasets
!pip install -q datasets==2.19.2


Found existing installation: datasets 2.21.0
Uninstalling datasets-2.21.0:
  Successfully uninstalled datasets-2.21.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.1 which is incompatible.


In [ ]:
import os, shutil

cache_path = os.path.expanduser("~/.cache/huggingface/datasets")
# tatoeba_mt cache klasörlerini sil (güvenli)
for name in os.listdir(cache_path):
    if "tatoeba_mt" in name.lower():
        shutil.rmtree(os.path.join(cache_path, name), ignore_errors=True)

print("Cache cleaned.")


Cache cleaned.


In [ ]:
from datasets import load_dataset

ds = load_dataset(
    "Helsinki-NLP/tatoeba_mt",
    "eng-spa",
    trust_remote_code=True,
    download_mode="force_redownload",
    verification_mode="no_checks"
)

print(ds)

# train yoksa validation kullan
base_split = "train" if "train" in ds else "validation"
base = ds[base_split]

print("Using split:", base_split)
print("Base size:", len(base))
print("Test size:", len(ds["test"]) if "test" in ds else "N/A")


Generating test split:   0%|          | 0/16582 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/197298 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['sourceLang', 'targetlang', 'sourceString', 'targetString'],
        num_rows: 16582
    })
    validation: Dataset({
        features: ['sourceLang', 'targetlang', 'sourceString', 'targetString'],
        num_rows: 197298
    })
})
Using split: validation
Base size: 197298
Test size: 16582


In [ ]:
# hangi splitler var?
print(ds.keys())

# train yoksa validation kullan
base = ds["train"] if "train" in ds else ds["validation"]

print("Using split:", "train" if "train" in ds else "validation")
print("Base size:", len(base))


dict_keys(['test', 'validation'])
Using split: validation
Base size: 197298


In [ ]:
N = 1000
subset = base.shuffle(seed=42).select(range(N))

# eng-spa config olduğundan:
# sourceString = English, targetString = Spanish
# biz ES->EN istiyoruz => src=Spanish(targetString), tgt=English(sourceString)

def preprocess(ex):
    return {
        "src": ex["targetString"].strip(),  # Spanish
        "tgt": ex["sourceString"].strip()   # English
    }

processed = subset.map(preprocess, remove_columns=subset.column_names)

print(processed[0])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'src': 'Él tomó mi paraguas sin molestarse en preguntar.', 'tgt': 'He took my umbrella without bothering to ask.'}


In [ ]:
for i in range(5):
    print("\nSRC:", processed[i]["src"])
    print("TGT:", processed[i]["tgt"])



SRC: Él tomó mi paraguas sin molestarse en preguntar.
TGT: He took my umbrella without bothering to ask.

SRC: Tom la salvó del incendio.
TGT: Tom saved her from the fire.

SRC: Un virus infectó la computadora de Tomás.
TGT: A virus infected Tom's computer.

SRC: Los científicos más exitosos son los que plantean las preguntas correctas.
TGT: The most successful scientists are those who pose the right questions.

SRC: Espero que apruebe el examen fácilmente.
TGT: I expect her to pass the examination easily.


In [ ]:
!pip install -q transformers sentencepiece accelerate


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-es-en"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Device:", device)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Device: cpu


In [ ]:
def translate_one(text, max_len=128, num_beams=4):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_length=max_len, num_beams=num_beams)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# hızlı kontrol
for i in range(5):
    src = processed[i]["src"]
    ref = processed[i]["tgt"]
    pred = translate_one(src)
    print("\nSRC:", src)
    print("PRED:", pred)
    print("REF:", ref)



SRC: Él tomó mi paraguas sin molestarse en preguntar.
PRED: He took my umbrella without bothering to ask.
REF: He took my umbrella without bothering to ask.

SRC: Tom la salvó del incendio.
PRED: Tom saved her from the fire.
REF: Tom saved her from the fire.

SRC: Un virus infectó la computadora de Tomás.
PRED: A virus infected Tomás' computer.
REF: A virus infected Tom's computer.

SRC: Los científicos más exitosos son los que plantean las preguntas correctas.
PRED: The most successful scientists are those who ask the right questions.
REF: The most successful scientists are those who pose the right questions.

SRC: Espero que apruebe el examen fácilmente.
PRED: I hope you pass the exam easily.
REF: I expect her to pass the examination easily.


In [ ]:
from tqdm.auto import tqdm

def translate_batch(texts, batch_size=16, max_len=128, num_beams=4):
    preds = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len).to(device)
        with torch.no_grad():
            out = model.generate(**inputs, max_length=max_len, num_beams=num_beams)
        preds.extend(tokenizer.batch_decode(out, skip_special_tokens=True))
    return preds

src_texts = [processed[i]["src"] for i in range(len(processed))]
refs = [processed[i]["tgt"] for i in range(len(processed))]

preds = translate_batch(src_texts, batch_size=16)
print("Preds:", len(preds), "Refs:", len(refs))
print("\nExample:\n", src_texts[0], "\n=>", preds[0])


  0%|          | 0/63 [00:00<?, ?it/s]

Preds: 1000 Refs: 1000

Example:
 Él tomó mi paraguas sin molestarse en preguntar. 
=> He took my umbrella without bothering to ask.


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "src_es": src_texts,
    "ref_en": refs,
    "pred_marian_es_en": preds
})

df.to_csv("marian_es_en_preds.csv", index=False, encoding="utf-8")
df.head()


,src_es,ref_en,pred_marian_es_en
0,Él tomó mi paraguas sin molestarse en preguntar.,He took my umbrella without bothering to ask.,He took my umbrella without bothering to ask.
1,Tom la salvó del incendio.,Tom saved her from the fire.,Tom saved her from the fire.
2,Un virus infectó la computadora de Tomás.,A virus infected Tom's computer.,A virus infected Tomás' computer.
3,Los científicos más exitosos son los que plant...,The most successful scientists are those who p...,The most successful scientists are those who a...
4,Espero que apruebe el examen fácilmente.,I expect her to pass the examination easily.,I hope you pass the exam easily.


In [ ]:
!pip install -q sacrebleu


In [ ]:
import sacrebleu

bleu = sacrebleu.corpus_bleu(preds, [refs])
print("BLEU score:", bleu.score)


BLEU score: 58.88649932752679


In [ ]:
!pip install -q nltk


In [ ]:
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

from nltk.translate.meteor_score import meteor_score

meteor_scores = [
    meteor_score([ref.split()], pred.split())
    for ref, pred in zip(refs, preds)
]

meteor_avg = sum(meteor_scores) / len(meteor_scores)
print("METEOR (avg):", meteor_avg)
print("METEOR (%):", meteor_avg * 100)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR (avg): 0.7425028266888046
METEOR (%): 74.25028266888046


In [ ]:
!pip install -q bert-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
from bert_score import score

P, R, F1 = score(
    preds,
    refs,
    lang="en",
    verbose=True
)

print("BERTScore F1 (avg):", F1.mean().item())


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/26 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 193.76 seconds, 5.16 sentences/sec
BERTScore F1 (avg): 0.9774094820022583
